In [111]:
from bs4 import BeautifulSoup as sp, XMLParsedAsHTMLWarning
import requests
import json
import warnings

warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)


In [112]:
sitemaps = [
    "https://www.parcasepeti.com.tr/product-sitemap1.xml",
    "https://www.parcasepeti.com.tr/product-sitemap2.xml",
    "https://www.parcasepeti.com.tr/product-sitemap3.xml",
    "https://www.parcasepeti.com.tr/product-sitemap4.xml",
    "https://www.parcasepeti.com.tr/product-sitemap5.xml",
]

final_links = []

for sitemap in sitemaps:
    response = requests.get(sitemap)
    soup = sp(response.text, "lxml")  # using XML parser

    loc_tags = soup.find_all("loc")
    for tag in loc_tags:
        final_links.append(tag.text.strip())

In [115]:
def get_data(link: str) -> dict:
    r = requests.get(link)
    soup = sp(r.text, "lxml")

    container = soup.find("div", class_="summary")
    product_id = container.find(class_ = 'wd-product-info').get("data-product-id")
    product_title = container.find(class_ = 'product_title').text

    product_info_raw = str(container.find(class_='woocommerce-product-details__short-description').find("p")).split(" : ")
    product_code = product_info_raw[1].strip().replace("</p>", "").split("  ")[0]
    product_partnumber = product_info_raw[-1].strip().replace("</p>", "")
    product_price = container.find(class_ = 'woocommerce-Price-amount amount').text
    product_stock = container.find("p", class_ = "stock").text.strip().lower()
    if product_stock == "stokta":
        product_stock = True
    else :
        product_stock = False

    product_images = []
    product_images_container = soup.find(class_ = "woocommerce-product-gallery__wrapper").find_all("figure")
    for product_image in product_images_container:
        image_src = product_image.get("data-thumb")
        product_images.append(image_src)

    product_description_html = str(soup.find("div", {"data-accordion-index": "description"})).replace("  ", "")

    product_attributes = []
    product_attributes_container = soup.find("table", {"class": "woocommerce-product-attributes"}).find_all("tr")
    for attribute in product_attributes_container:
        attribute_title = attribute.find("th").text.replace("\n", "").strip()
        attribute_value = attribute.find("td").text.replace("\n", "").strip()
        product_attributes.append({
            "attribute_title": attribute_title,
            "attribute_value": attribute_value,
        })

    product_info = {
        "product_id": product_id,
        "product_title": product_title,
        "product_code": product_code,
        "product_partnumber": product_partnumber,
        "product_price": product_price,
        "product_stock": product_stock,
        "product_description_html": product_description_html,
        "product_attributes": product_attributes,
        "product_images": product_images,
    }

    return product_info


In [116]:
for enum, link in enumerate(final_links):
    print(f"[{enum}/{len(final_links)}] {link}]")
    try:
        product_info = get_data(link)
        with open("completed.txt", "a") as file:
            file.write(link + "\n")
        print(f"\t{product_info.get("product_title")} - encontrado")
    except Exception as e:
        with open("error.txt", "a") as file:
            file.write(link + "\n")
        print(f"\tErro: {link}")
        continue

    with open("final.json", "a") as file:
        file.write(json.dumps(product_info))

[0/21506] https://www.parcasepeti.com.tr/magaza/]
	Erro: https://www.parcasepeti.com.tr/magaza/
[1/21506] https://www.parcasepeti.com.tr/urun/turbo-hortumu-c3-1-4-hdi-16v-dv4ted4-144036/]
	
Turbo Hortumu C3 1.4 Hdi 16V Dv4Ted4 144036 - encontrado
[2/21506] https://www.parcasepeti.com.tr/urun/vakum-pompasi-301-c-elysee-4556-24-av6q2a454aa/]
	
Vakum Pompası 301 / C-Elysee 4556.24 AV6Q2A454AA - encontrado
[3/21506] https://www.parcasepeti.com.tr/urun/turbo-hortumu-208-c3-2008-3008-9835265880-9835737580/]
	
Turbo Hortumu 208-C3-2008-3008 9835265880 9835737580 - encontrado
[4/21506] https://www.parcasepeti.com.tr/urun/turbo-borusu-partner-berlingo-1-6-hdi-1434-g3-9684362180/]
	
Turbo Borusu Partner Berlingo 1.6 Hdi 1434.G3 9684362180 - encontrado
[5/21506] https://www.parcasepeti.com.tr/urun/turbo-borusu-contasi-301-c-elysee-partner-tepee-berlingo-3-9804177480/]
	
Turbo Borusu Contası 301 C-Elysee Partner Tepee Berlingo 3 9804177480 - encontrado
[6/21506] https://www.parcasepeti.com.tr/urun


KeyboardInterrupt

